# PC setting

In [1]:
# !pip install -r requirements_uavUnet.txt
from google.colab import drive
drive.mount('/content/drive')

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_properties = torch.cuda.get_device_properties(device)
    gpu_memory_size = gpu_properties.total_memory / 1024 ** 3  # Convert bytes to gigabytes
    print(f"GPU Memory Size: {gpu_memory_size:.2f} GB")
else:
    print("GPU is not available.")


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/drive/MyDrive/Forest/src/

# Import modules

In [25]:


import os, glob, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import random
from PIL import Image

# print(f"GPU Memory Usage: {get_gpu_memory_usage():.2f} GiB\n")
# print("Allocated Tensors:")
# print_allocated_tensors()

import segment_model_training
import importlib
importlib.reload(segment_model_training)
import cv2
from matplotlib import pyplot as plt

from segment_model_training import calculate_statistics,LoadDataSet,get_train_transform
from fpathutils import copyLocaliImages, get_mskPath
# importlib.reload(argmentDataset)
# importlib.reload(get_train_transform)

GPU Memory Size: 16.00 GB


# LoadDatasets

In [26]:

datasetName = "Forest tsumura 2 50m P4Pv2"
treeType = "cypress"
cropSize = 1024
lapSize=512

# orgDir = f"03_datasetforModel\\{datasetName}_{treeType}\\org_crop4Corner_5120_3072_Size{cropSize}_lap{lapSize}".replace("\\","/")
orgDir = f"C:\\datas\\uav_cnn_cypress\\test_org_crop4Corner_5120_3072_Size1024_lap512"
orgPaths = glob.glob(orgDir + "/*.*")

len(orgPaths)

# フォルダのパスを指定して統計量を計算
# mean_values, std_deviation = calculate_statistics(orgDir)
mean_values, std_deviation = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
print("正規化ずみ平均値:", mean_values)
print("正規化ずみ標準偏差:", std_deviation)


正規化ずみ平均値: (0.485, 0.456, 0.406)
正規化ずみ標準偏差: (0.229, 0.224, 0.225)


In [27]:

# calculate_statistics(orgPaths)

In [28]:

orgDataset_count = len(glob.glob(orgDir + "/*.*"))

rotate_270per90 = True
flipMirror = True

# orgDir = segment_model_training.argmentDataset(orgDir, rotate=rotate_270per90, flipMirror=True)
# segment_model_training.main("cedar",2)


workDir = "./04_trainingModel"
TRAIN_PATH = workDir + f'/{datasetName}_{treeType}/'
os.makedirs(TRAIN_PATH, exist_ok=True)
orgDataset_count

28

In [29]:

orgPaths = glob.glob(orgDir + "/*.*")
img = Image.open(orgPaths[0])
mskPaths = [get_mskPath(orgPath) for orgPath in orgPaths]
print(len(orgPaths),len(mskPaths))

# orgDir = copyLocaliImages(orgDir, f"C:\\datas\\uav_cnn_{treeType}")
# orgPaths = glob.glob(orgDir + "/*.*")


print(orgDir)

28 28
C:\datas\uav_cnn_cypress\test_org_crop4Corner_5120_3072_Size1024_lap512


# PSQL connect

In [30]:
# import psql_connect
# importlib.reload(psql_connect)

# engine, conn, engineSQL = psql_connect.getPSQL(databaseName="cnn_segment", port="5432", ownerName="matsuilocal")

# psql_connect.createTB_UnetDatasetUAV()
# psql_connect.createTB_UnetTrainingUAV()

In [31]:
# from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData
# from sqlalchemy.orm import sessionmaker

# # Define the metadata and the table
# metadata = MetaData()
# unet_dataset_uav = Table('unet_dataset_uav', metadata,
#                          Column('id', Integer, primary_key=True),
#                          Column('datasetName', String),
#                          Column('classType', String),
#                          Column('width', Integer),
#                          Column('height', Integer),
#                          Column('lapSize', Integer),
#                          Column('original_dataset_count', Integer),
#                          Column('argment_rotate', String),
#                          Column('argment_flipMirror', String),
#                          Column('datasetDir', String),
#                          Column('dataset_count', Integer),
#                          Column('mean', String),
#                          Column('std', String),
#                      )
# import ast

# select_SQL = "select * from unet_dataset_uav;"

# df_db = pd.read_sql(sql=select_SQL, con=engineSQL)
# if not str(list(mean_values)) in list(df_db["mean"]):
# # mean_values_fromDB = ast.literal_eval(df_db.loc[0,"mean"])
#     # Create a session
#     Session = sessionmaker(bind=engine)
#     session = Session()

#     # Create a new row to insert
#     new_row = unet_dataset_uav.insert().values(
#         datasetName=datasetName,
#         classType=treeType,
#         width=img.width,
#         height=img.height,
#         lapSize=lapSize,
#         original_dataset_count=orgDataset_count,
#         argment_rotate=rotate_270per90,
#         argment_flipMirror=flipMirror,
#         datasetDir=orgDir,
#         dataset_count=len(orgPaths),
#         mean=str(list(mean_values)),
#         std=str(list(std_deviation)),
#     )
#     print(new_row)

#     try:
#         # Execute the insert statement
#         session.execute(new_row)

#         # Commit the changes
#         session.commit()

#         # Close the session
#         session.close()
#     except Exception as e:
#         print(e)
#         session.close()

In [32]:
# datasetID = int(df_db["id"][0])


# getDatasetParam_SQL = f"select * from unet_dataset_uav where id={4};"
# df = pd.read_sql(getDatasetParam_SQL,engine)
# df

In [33]:
# orgDir = df["datasetDir"].iloc[0]
# orgDir

# Dataset define

In [62]:
from segment_model_training import printArrayStatics

import segment_model_training
importlib.reload(segment_model_training)

print(orgDir.replace("\\","\\\\"))
train_dataset = segment_model_training.LoadDataSet(orgDir)
# train_dataset = LoadDataSet(orgDir.replace("\\","\\\\")+"\\")
print("images count\t",train_dataset.__len__())

org, msk = train_dataset.__getitem__(4)
print(org.dtype, msk.dtype)
printArrayStatics(org)
printArrayStatics(msk)

segment_model_training.printArrayStatics(org)
segment_model_training.printArrayStatics(msk)

BATCHSIZE = 1

#データ前処理
split_ratio = 0.3
train_size=int(np.round(train_dataset.__len__()*(1 - split_ratio),0))
valid_size=int(np.round(train_dataset.__len__()*split_ratio,0))

print(f"train count : {(train_size)}")
print(f"valid count : {(valid_size)}")


train_data, valid_data = random_split(train_dataset, [train_size, valid_size])
train_loader = DataLoader(dataset=train_data, batch_size=BATCHSIZE, shuffle=True)
val_loader = DataLoader(dataset=valid_data, batch_size=BATCHSIZE, shuffle=True)


GPU Memory Size: 16.00 GB
C:\\datas\\uav_cnn_cypress\\test_org_crop4Corner_5120_3072_Size1024_lap512
images count	 28
(2, 1024, 1024)
mask torch.Size([2, 1024, 1024, 1])
torch.float32 torch.float32
torch.Size([3, 1024, 1024]) tensor(1.) tensor(0.3769) tensor(0.)
tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196])
torch.Size([2, 1024, 1024, 1]) tensor(1.) tensor(0.5000) tensor(0.)
tensor([0., 1.])
torch.Size([3, 1024, 1024]) tensor(1.) tensor(0.3769) tensor(0.)
tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196])
torch.Size([2, 1024, 1024, 1]) tensor(1.) tensor(0.5000) tensor(0.)
tensor([0., 1.])
train count : 20
valid count : 8


# Models module Load

In [63]:
# !pip install segmentation_models_pytorch
from torch import nn
from unet_model import UNet, DiceBCELoss, save_ckp, load_ckp
from pre_segmentation_model import UnetModel, calculate_iou, validateModel
import segmentation_models_pytorch as smp

import pre_segmentation_model
importlib.reload(pre_segmentation_model)


encoder_name = "resnet34"
encoder_weight = "imagenet"
DEVICE = "cuda"
num_epochs = 100

model = pre_segmentation_model.UnetModel(encoder_name, encoder_weight, in_ch=3, outClass=2, decoder_channels=[256, 128, 64, 32, 16], activationName="sigmoid")
modelID = "class-{}_data{}_batch{}_epoch{}_model{}".format(treeType,train_dataset.__len__(), BATCHSIZE, num_epochs, encoder_name + "-" + encoder_weight)
print(modelID)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def init_weights(module):
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.ConvTranspose2d):
        # 畳み込み層や転置畳み込み層の重みを初期化
        nn.init.xavier_uniform_(module.weight.data)
        if module.bias is not None:
            nn.init.constant_(module.bias.data, 0)
    elif isinstance(module, nn.BatchNorm2d):
        # バッチ正規化層の重みを初期化
        nn.init.constant_(module.weight.data, 1)
        nn.init.constant_(module.bias.data, 0)

model.apply(init_weights)

# optimizer = torch.optim.Adam([dict(model.parameters(),lr = 1e-3))])
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=1e-9)])
# 損失関数を定義します。セグメンテーションタスクのためにCrossEntropyLossを使用します。
# criterion = nn.CrossEntropyLoss()

# criterion = smp.utils.losses.DiceLoss()
criterion = smp.utils.losses.BCEWithLogitsLoss()

accuracy_metric = smp.utils.metrics.IoU(threshold=0.5)
metrics = [accuracy_metric]
# valid_loss_min = np.Inf


torch.cuda.empty_cache()
# CuDNNを使用したネットワークのベンチマークを有効にし、パフォーマンスを最適化します。
torch.backends.cudnn.benchmark = True

train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=criterion, 
    metrics=metrics, 
    device=DEVICE
)


# os.makedirs(workDir, exist_ok=True)

# データローダー（train_loader, val_loaderなど）を定義した後、訓練ループを開始します。
for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        
        printArrayStatics(masks)
        
        # 前方計算を行います。
        predicts = model(images)
        
        # print(predicts.shape, masks.shape)
        # print("\tPred,mask",predicts.cpu().min(), predicts.cpu().max(), masks.cpu().min(), masks.cpu().max())
        # predicts[predicts>=0.5] = 1.0

        # 損失を計算します。
        loss = criterion(predicts, masks)
        batch_iou = metrics[0](predicts, masks)
        print("Epoch",epoch,"\tLoss", loss.unique(),"\tIoU",batch_iou.unique())
        
        # 勾配を初期化します。
        optimizer.zero_grad()

        # バックプロパゲーションを行います。
        loss.backward()

        # オプティマイザを更新します。
        optimizer.step()

    # バリデーションデータでモデルのパフォーマンスを評価します。
    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            # 前方計算を行います。
            predicts = model(images)
            
            # print(predicts.unique())
            pred_show = predicts[0]
            pred_show = (pred_show >= 0.5).float()
            # print(pred_show.unique())
            
            segment_model_training.savePred(img_show=images[0],
                                            msk_show=masks[0],
                                            pred=pred_show,
                                            # mean_values=mean_values,
                                            # std_deviation=std_deviation,
                                            mean_values=[0,0,0],
                                            std_deviation=[0,0,0],
                                            workDir=workDir,
                                            epoch=epoch,
                                            modelID=modelID)

            pred_show = predicts[1]
            pred_show = (pred_show >= 0.5).float()
            # print(pred_show.unique())
            
            segment_model_training.savePred(img_show=images[1],
                                            msk_show=masks[1],
                                            pred=pred_show,
                                            # mean_values=mean_values,
                                            # std_deviation=std_deviation,
                                            mean_values=[0,0,0],
                                            std_deviation=[0,0,0],
                                            workDir=workDir,
                                            epoch=epoch,
                                            modelID=modelID)
            
            # ここで必要なら評価指標を計算します。

# モデルを保存します。
torch.save(model.state_dict(),os.path.join(workDir,'mworkDirpth'))

class-cypress_data28_batch1_epoch100_modelresnet34-imagenet


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 16.00 GiB total capacity; 15.10 GiB already allocated; 0 bytes free; 15.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
checkpoint_path = os.path.join(workDir,f'model_{treeType}_chkpoint_')
best_model_path = os.path.join(workDir,f'model_{treeType}_bestmodel.pt')
os.makedirs(os.path.dirname(best_model_path), exist_ok=True)
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

print(checkpoint_path)
print(best_model_path)

total_train_loss = []
total_train_score = []
total_valid_loss = []
total_valid_score = []
losses_value = 0


def get_gpu_memory_usage():
    """GPUメモリの使用量を取得する関数"""
    return torch.cuda.memory_allocated() / 1024**3  # GiB単位で返す

def print_allocated_tensors():
    """GPUメモリを占有している変数の一覧を表示する関数"""
    tensor_list = torch.cuda.memory_summary()  # 現在GPUメモリに割り当てられているテンソルの一覧を取得
    print(tensor_list)

# 使用例
print(f"GPU Memory Usage: {get_gpu_memory_usage():.2f} GiB\n")
# print_allocated_tensors()

./04_trainingModel\class-cypress_data28_batch2_epoch100_modelresnet34-imagenet\model_cypress_chkpoint_
./04_trainingModel\class-cypress_data28_batch2_epoch100_modelresnet34-imagenet\model_cypress_bestmodel.pt
GPU Memory Usage: 3.52 GiB



In [20]:
patience = 5 # 5Epcoch以上連続でモデル精度が向上しなければEarly Stopping
early_stop_counter = 0

from pprint import pprint

max_score = 0
for i in range(num_epochs):
    
    print(f"Epoch:{i+1}")
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(val_loader)
    
    pprint(train_logs)
    pprint(valid_logs)
    
    # IoUスコアが最高値が更新されればモデルを保存
    if max_score < valid_logs["iou_score"]:
        max_score = valid_logs["iou_score"]
        torch.save(model, checkpoint_path)
        print("Model saved!")
        early_stop_counter = 0

    else:
        early_stop_counter += 1
        print(f"not improve for {early_stop_counter}Epoch")
        if early_stop_counter==patience:
            print(f"early stop. Max Score {max_score}")
            break

    # 適当なタイミングでlearning rateの変更
    if i % 10==0:
        optimizer.param_groups[0]["lr"] = 1e-5
        print("Decrease decoder learning rate to 1e-5")

Epoch:1
valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  8.64it/s, bce_with_logits_loss - 0.8908, iou_score - 0.09547]
{'bce_with_logits_loss': 0.8058724701404572, 'iou_score': 0.24309500306845738}
{'bce_with_logits_loss': 0.890825092792511, 'iou_score': 0.09547429636592669}
Model saved!
Decrease decoder learning rate to 1e-5
Epoch:2
valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  8.24it/s, bce_with_logits_loss - 0.8686, iou_score - 0.118]
{'bce_with_logits_loss': 0.8024658262729645, 'iou_score': 0.25825796164573256}
{'bce_with_logits_loss': 0.8685535937547684, 'iou_score': 0.11801002954606246}
Model saved!
Epoch:3
valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  8.42it/s, bce_with_logits_loss - 0.8571, i

In [16]:
workDir = "./04_trainingModel"

from segment_model_training import showPred
importlib.reload(segment_model_training)



workDir = os.path.join(workDir, modelID) + "\\"
os.makedirs(workDir, exist_ok=True)

img_show, msk_show = random.choice(valid_data)
# img_show[img_show<=0]=0
print("0 org_test.unique()",np.unique(img_show)[:50]) 
print("0 msk_show.unique()",np.unique(msk_show)) 


img_pred = torch.stack([org, org])
# pred = model(img_show.cuda())
pred = torch.sigmoid(model(img_pred.cuda()))
print("0 pred_sig.unique()",np.unique(pred.data.cpu())) 
pred = pred[0][0]
print("predShape",pred.shape,pred.data.cpu().dtype)
print(msk_show.dtype)
print("0 pred.unique()",np.unique(pred.data.cpu())) 
img_show = img_show.cuda()

print(workDir)
print(modelID)
segment_model_training.showPred(img_show=img_show, msk_show=msk_show, pred=pred,
         mean_values=mean_values, std_deviation=std_deviation,
         workDir=workDir, epoch=1, modelID="testmodel",
         imgSave=True)    
print(org.shape)


# pred = model(org)[0][0]
# pred.shape

GPU Memory Size: 16.00 GB
0 org_test.unique() [0.         0.00416667 0.00833333 0.0125     0.01666667 0.02083333
 0.025      0.02916667 0.03333334 0.0375     0.04166667 0.04583333
 0.05       0.05416667 0.05833333 0.0625     0.06666667 0.07083333
 0.075      0.07916667 0.08333334 0.0875     0.09166667 0.09583333
 0.1        0.10416666 0.10833333 0.1125     0.11666667 0.12083333
 0.125      0.12916666 0.13333334 0.1375     0.14166667 0.14583333
 0.15       0.15416667 0.15833333 0.1625     0.16666667 0.17083333
 0.175      0.17916666 0.18333334 0.1875     0.19166666 0.19583334
 0.2        0.20416667]
0 msk_show.unique() [0.]
0 pred_sig.unique() [0.50011766 0.50017285 0.5003491  ... 0.7278361  0.7278646  0.72880834]
predShape torch.Size([1024, 1024]) torch.float32
torch.float32
0 pred.unique() [0.50011766 0.50017285 0.5003491  ... 0.7278361  0.7278646  0.72880834]
./04_trainingModel\class-cypress_data28_batch2_epoch100_modelresnet34-imagenet\
class-cypress_data28_batch2_epoch100_modelresn

In [ ]:
data = {'dataset_Mean': mean_values, 'dataset_Std Deviation': std_deviation}
df_statics = pd.DataFrame(data)
# DataFrameをCSVファイルとして保存
df_statics.to_csv(workDir + f"statistics_{modelID}.csv", index=False)


In [ ]:
# del pre_segmentation_model
import pre_segmentation_model
importlib.reload(pre_segmentation_model)

<module 'pre_segmentation_model' from 'H:\\マイドライブ\\Forest\\src\\pre_segmentation_model.py'>

In [38]:
num_epochs = 20

for epoch in range(num_epochs):
    print(f"Epoch {str(epoch+1)}/{str(num_epochs)}")

#<---------------トレーニング---------------------->
    train_loss = []
    train_score = []
    valid_loss = []
    valid_score = []
    train_loader_pbar = tqdm(train_loader, desc = 'description')
    valid_loss_min = 0
    
    #<---------------評価---------------------->
    losses_value, score = pre_segmentation_model.validateModel(model, train_loader_pbar, workDir=workDir,epoch=epoch,means=mean_values,stds=std_deviation, modelID=modelID)

    train_loss.append(losses_value)
    train_score.append(score)
    
    train_loader_pbar.set_description(f"Epoch: {epoch+1}, loss: {losses_value}, IoU: {score}")
    
    total_train_loss.append(np.mean(train_loss))
    total_train_score.append(np.mean(train_score))
    total_valid_loss.append(np.mean(valid_loss))
    total_valid_score.append(np.mean(valid_score))
    print(f"Train Loss: {total_train_loss[-1]}, Train IOU: {total_train_score[-1]}")
    print(f"Valid Loss: {total_valid_loss[-1]}, Valid IOU: {total_valid_score[-1]}")

    
    checkpoint = {
        'epoch': epoch + 1,
        'valid_loss_min': total_valid_loss[-1],
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    
    print("CheckPoint Save",modelID)

    # checkpointの保存
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    
    # 評価データにおいて最高精度のモデルのcheckpointの保存
    if total_valid_loss[-1] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,total_valid_loss[-1]))
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = total_valid_loss[-1]
    
    print("")


    score = {
        # "epoch" : range(1,num_epochs+1),
        "train_Loss" : total_train_loss,
        "valid__Loss" : total_valid_loss,
        "train_scoreIoU" : total_train_score,
        "valid__scoreIoU" : total_valid_score,
        }

    import pandas as pd
    df_score = pd.DataFrame(score, index=range(1,len(total_train_loss)+1))
    df_score.to_csv(workDir + f"scoreSheet_{modelID}.csv")




Epoch 1/20


description: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]


Average Loss: 0.9117
Average IoU: 0.2905
Train Loss: 0.9116533696651459, Train IOU: 0.2905269622802782
Valid Loss: nan, Valid IOU: nan
CheckPoint Save class-cypress_data28_batch2_epoch100_modelresnet34-imagenet

Epoch 2/20


description: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.96it/s]


Average Loss: 0.9117
Average IoU: 0.2905
Train Loss: 0.911653357744217, Train IOU: 0.2905269622802782
Valid Loss: nan, Valid IOU: nan
CheckPoint Save class-cypress_data28_batch2_epoch100_modelresnet34-imagenet

Epoch 3/20


...

Average Loss: 0.9117
Average IoU: 0.2905
Train Loss: 0.91165332198143, Train IOU: 0.2905269622802877
Valid Loss: nan, Valid IOU: nan
CheckPoint Save class-cypress_data28_batch2_epoch100_modelresnet34-imagenet

Epoch 19/20


description: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]


Average Loss: 0.9117
Average IoU: 0.2905
Train Loss: 0.911653333902359, Train IOU: 0.29052696228028296
Valid Loss: nan, Valid IOU: nan
CheckPoint Save class-cypress_data28_batch2_epoch100_modelresnet34-imagenet

Epoch 20/20


description: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]


Average Loss: 0.9117
Average IoU: 0.2905
Train Loss: 0.9116533517837524, Train IOU: 0.2905269622802877
Valid Loss: nan, Valid IOU: nan
CheckPoint Save class-cypress_data28_batch2_epoch100_modelresnet34-imagenet

